# Rag com LCEL

Como fazer o processo de RAG. 

## LangChain Expression Language (LCEL)

A Linguagem de Expressão LangChain, ou LCEL, é uma forma declarativa de compor cadeias de maneira fácil. A LCEL foi projetada desde o primeiro dia para suportar a colocação de protótipos em produção, sem a necessidade de alterações no código, desde a cadeia mais simples "prompt + LLM" até as cadeias mais complexas (já vimos pessoas executando com sucesso cadeias LCEL com centenas de estapas em produção). Para destacar alguns dos motivos pelos quais você pode querer usar a LCEL:
- **Suporte a streaming de primeira classe**: menor tempo possível para saída do primeiro token produzido.
- **Suporte Assíncrono**: Qualquer cadeia construída co a LCEL, pode ser chamada tanto com a API assíncrona;
- **Execução Paralela otimizada**: Sempre que suas cadeias LCEL tiverem etapas que podem ser executadas em paralelo, automaticamente é feito isso. 
- **Retentativas e fallbacks**: É maneira de tornar suas cadeias mais confiáveis em grande escala, na qual ações alterantivas odem ser tomadasa no aso de um erro em uma cadeia.
- **Acessar resultados intermediáveios**: Auxiliando na depuração de uma cadeia.

In [19]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [20]:
caminhos = [
  'arquivos/E-Book Python Avançado.pdf',
]

paginas = []

for caminho in caminhos:
  loader = PyPDFLoader(caminho)
  paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
  chunk_size=500,
  chunk_overlap=50,
  separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

In [21]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [22]:
from langchain_community.vectorstores.faiss import FAISS

vectorstore= FAISS.from_documents(
  documents=documents,
  embedding=embeddings_model
)

retriever = vectorstore.as_retriever(search_type='mmr')

Agora na criação da chain é que temos uma mudança. Podemos definir nossa chain manualmente da seguinte forma:

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()

template_str = """ Responda as seguintes perguntas do usuário utilizando paenas o contexto fornecido
Contexto: {contexto}

Pergunta: {pergunta}
"""

template = ChatPromptTemplate.from_template(template_str)
outputParser = StrOutputParser() 

In [24]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_and_retrieval = RunnableParallel(
  {
    'pergunta': RunnablePassthrough(), 
    'contexto': retriever
  }
)

chain = setup_and_retrieval | template | model | outputParser

In [25]:
chain.invoke("O que é python?")

'Python é uma linguagem de programação interpretada, onde todas as variáveis são objetos. É uma linguagem de alto nível que permite a reutilização de códigos e é muito popular entre os desenvolvedores.'

# Entendendo o RunnableParallel e o RunnablePassTrough

In [28]:
{ 'Pergunta': RunnablePassthrough().invoke('O que é python?')}

{'Pergunta': 'O que é python?'}

In [29]:
retriever.invoke('O que é python')

[Document(page_content='presente curso visa apresentar conceitos avançados da linguagem Python.\nDevido à grande popularidade do Python, seu aprendizado por parte dos\ndesenvolvedores pode ser um grande diferencial.\nBons estudos!\nMarcos Roberto Ribeiro', metadata={'source': 'arquivos/E-Book Python Avançado.pdf', 'page': 6}),
 Document(page_content='de reutilização de códigos. Na POO, os programas são compostos por objetos que, por\nsua vez, possuem atributos e métodos. Os atributos são como variáveis inerentes ao\nobjetos para representar suas características. Já os métodos são funções específicas dos\nobjetos (CEDER, 2018).\nNa linguagem Python, todas as variáveis são objetos. Como exemplo, se tivermos\numa variável  nome do tipo  str, a instrução  nome.lower() chama o método  lower() do', metadata={'source': 'arquivos/E-Book Python Avançado.pdf', 'page': 55}),
 Document(page_content='elementos a serem escritos na tela.\n1.3 Operadores e expressões\nAssim como a maioria das linguage

In [30]:
RunnableParallel(
  {
    'pergunta': RunnablePassthrough(),
    'contexto': retriever
  }
).invoke('O que é python?')

{'pergunt': 'O que é python?',
 'contexto': [Document(page_content='presente curso visa apresentar conceitos avançados da linguagem Python.\nDevido à grande popularidade do Python, seu aprendizado por parte dos\ndesenvolvedores pode ser um grande diferencial.\nBons estudos!\nMarcos Roberto Ribeiro', metadata={'source': 'arquivos/E-Book Python Avançado.pdf', 'page': 6}),
  Document(page_content='de reutilização de códigos. Na POO, os programas são compostos por objetos que, por\nsua vez, possuem atributos e métodos. Os atributos são como variáveis inerentes ao\nobjetos para representar suas características. Já os métodos são funções específicas dos\nobjetos (CEDER, 2018).\nNa linguagem Python, todas as variáveis são objetos. Como exemplo, se tivermos\numa variável  nome do tipo  str, a instrução  nome.lower() chama o método  lower() do', metadata={'source': 'arquivos/E-Book Python Avançado.pdf', 'page': 55}),
  Document(page_content='como editor de código, console, explorador de variáve

No final o que estamos fazendo ao utilizarmos o RunnableParallel é criar uma estrutura de dicionário cujo os valores são gerados em paralelo

# Uma alterantiva não paralelizável

Da seguinte forma, também obteriámos o mesmo resultado, mas de forma não paralelizada, o que pode gerar um atraso no processamento da nossa chain

In [32]:
setup_dict = {'pergunta': RunnablePassthrough(), 'contexto': retriever}
chain = setup_dict | template | model | outputParser
chain.invoke('O que é python?')

'Python é uma linguagem de programação interpretada, onde todas as variáveis são objetos.'

# Fallbalcks

Ao trabalhar com modelos de linguagem, você pode frequentemente encontrar problemas nas APIs subjacentes, seja por limitação de taxa ou tempo de inatividade. Portanto, à medida que você move suas aplicações LLM para produção, torna-se cada vez mais importante proteger-se contra esses problemas. É por isso que introduzimos o conceito de **fallbacks**, ou alternativas em português. 

Uma alternativa é um plano substituto que pode ser usado em uma emergência. 

Criticamente, as alternativas podem ser aplicadas não apenas no nível do LLM, mas em todo o nível executável. Isso é importante porque, muitas vezes, modelos diferentes exigem prompts diferentes. Então, se sua chamada para OpenAI falhar, você não quer simplesmente enviar o mesmo prompt para o Anthropic - você provavelmente vai querer usar um template de prompt diferente e enviar uma versão diferente lá.

## Fallback para entradas grandes 

Quando construímos aplicações, precisamos sempre atentar às questões economicas que envolvem colocar m modelo em produção.
Muitas vezes será necessário otimizar nossos custos, evitando utilizar modelos maiores (e consequentemente mais caros) para problemas simples. Com fallback, temos a alternativa de sempre tentar processar a entrada do usuário com um modelo menor e, caso tivermos um erro no processamento, tentamos com um modelo mais oneroso. Mostramos aqui um exemplo onde utilizamos um modelo mais simples com context window menor que, no caso de uma entrada maior, é substituído por um modelo mais complexo. 


In [34]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model='gpt-3.5-turbo-instruct')
prompt_str= """
  Resuma o seguinte texto: {texto}
"""
prompt = PromptTemplate.from_template(prompt_str)

chain_pequena = prompt | llm

chain_pequena.invoke({'texto': 'Oi, eu sou o Douglas'})

'\nO texto apresenta uma pessoa chamada Douglas, que se identifica como autor da mensagem. '

In [35]:
chain_pequena.invoke({'texto': 'Oi, eu sou o Douglas' * 1000})


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 6267 tokens (6011 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [45]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model =  ChatOpenAI(model='gpt-3.5-turbo-0125')
prompt = ChatPromptTemplate.from_template('Resuma o seguinte texto: {texto}')
outputParser = StrOutputParser()
chain_grande = prompt | model | outputParser

In [40]:
chain_grande.invoke({'texto': 'Oi, eu sou o Douglas' * 1000})

AIMessage(content='Oi, eu sou o Douglas.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 6015, 'total_tokens': 6022, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8f4475de-246c-4f89-a72e-ee0ce34e578a-0')

In [46]:
chain_fallback = chain_pequena.with_fallbacks([chain_grande])
chain_fallback.invoke({'texto': 'Oi, eu sou o Douglas' * 1000})

'Resumo: O texto repetitivo afirma várias vezes que o autor se chama Douglas.'

In [44]:
chain_fallback.invoke({'texto': 'Oi, eu sou o Douglas'})


'\n "Meu nome é Douglas." '

# Fallbacks para formatações

Da mesma forma ocorre na formatação de informação, aplicação muito corrente para modelos de linguagem. Em geral, podemos utilizar modelos simples para essas funções mas, seguidamente, pelas suas limitações eles acabam errando. Nestes casos, podemos utilizar fallbacks para gerar uma formatação com modelos mais complexos quando houver erros. 

In [47]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser

prompt = PromptTemplate.from_template(
  """Qual foi a data e hora do evento {evento} (no formato %y-%m-%dT%H:%M:%S.%fZ - retorne apenas o valor solicitado) """
)

In [51]:
chain_instruct = prompt | OpenAI(model='gpt-3.5-turbo-instruct') | DatetimeOutputParser()

chain_instruct.invoke({'evento': 'A final da copa do mundo de 2002'})

datetime.datetime(2002, 6, 30, 18, 0)

In [53]:
chain_turbo = prompt | ChatOpenAI(model='gpt-3.5-turbo') | DatetimeOutputParser()
chain_turbo.invoke({'evento': 'A final da copa do mundo de 2002'})

datetime.datetime(2002, 6, 30, 12, 0)

In [54]:
chain_fallback = chain_instruct.with_fallbacks([chain_turbo])
chain_fallback.invoke({'evento': 'A final da copa do mundo de 2002'})


datetime.datetime(2002, 6, 30, 0, 0)

# Principais métodos de chains com LCEL

A interface padrão de LCEL inclui os seguintes métodos:

- **stream**: transmitir de volta fragmentos da resposta
- **invoke**: chamar a cadeia com um input
- **batch**: chamar a cadeia com uma lista de inputs

Esses também possuem métodos assíncronos correspondentes que devem ser usados com a sintaxe **asyncio await** para concorrência:

- **astream**: transmitir de volta fragmentos da resposta de forma assíncrona
- **ainvoke**: chamar a cadeia com um input de forma assíncrona
- **abatch**: chamar a cadeia com uma lista de inputs de forma assíncrona
- **astream_log**: trasmitir de volt etapas intermediárias à medida que acontecem, além da resposta final
- **astream_events**: transmitir eventos beta à medida que acontecem na cadeia.

In [55]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model='gpt-3.5-turbo')
prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o assunto: {assunto}')

chain = prompt | model

# Invoke

O invoke é o método básico para inserir um input na cadeia e receber uma resposta

In [56]:
chain.invoke('Como é representado uma lista no python')

AIMessage(content='Uma lista em Python é representada por elementos separados por vírgulas e delimitados por colchetes, permitindo o armazenamento e manipulação de múltiplos valores de forma ordenada.', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 24, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-09ed7b4b-4471-4ef7-b01d-72dcb1b7be80-0')

Ele pode ser rodado como uma simples string quando existe apenas uma input no prompt, mas a forma mais recomendada é informando especificamente o nome da input através de um dicionário

In [57]:
chain.invoke({'assunto': 'Como é representado uma lista no python'})


AIMessage(content='Uma lista no Python é representada por uma sequência ordenada de elementos, podendo conter diferentes tipos de dados e sendo acessada através de índices numéricos.', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 24, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ae4712a5-f064-439d-9873-92c124a3f93b-0')

# Stream 

Para recebermos uma saída conforme ela é gerada pelo modelo utilizamos o stream.

In [59]:
for chunk in chain.stream({'assunto': 'Como é representado uma lista no python'}):
  print(chunk.content, end='', flush=True)

Uma lista no Python é representada por uma sequência ordenada de elementos, podendo conter diferentes tipos de dados e ser acessada através de índices numéricos.

# Batch

Para fazermos múltiplas requisições em paralelo utilizamos batch 

In [62]:
chain.batch([
  {'assunto': 'lista'},
  {'assunto': 'dicionário'},
  {'assunto': 'lambdas'}
], config={'max_concurrency':2})

[AIMessage(content='"Uma lista bem organizada pode ser a chave para o sucesso em diversas áreas da vida."', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 17, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-578e1810-a5cb-463e-a9cd-f51fd1b6c6ed-0'),
 AIMessage(content='O dicionário é o melhor amigo de quem busca enriquecer o vocabulário e ampliar o conhecimento linguístico.', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 19, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0

# Ainvoke

Todos os métodos possuem assíncronos correspondentes que devem ser usados com a sintaxe **asyncio await** para concorrência

In [63]:
import asyncio

async def process_chain(input):
  response = await chain.ainvoke(input)
  return response

In [68]:
from time import time 

a = time()

task1 = asyncio.create_task(process_chain({'assunto': 'list'}))
task2 = asyncio.create_task(process_chain({'assunto': 'array'}))
task3 = asyncio.create_task(process_chain({'assunto': 'string'}))

await task1
await task2
await task3

b = time()
print(a-b)

-1.1336848735809326


In [67]:
task2.result()

AIMessage(content='Um array é como uma caixa de ferramentas, cheia de elementos prontos para serem acessados e utilizados de forma organizada e eficiente.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 17, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ba0cd7f9-c8a0-41ac-88c0-b755e8999bed-0')

In [71]:
from time import time 

a = time()

task1 = chain.invoke({'assunto': 'list'})
task2 = chain.invoke({'assunto': 'array'})
task3 = chain.invoke({'assunto': 'string'})

b = time()
print(a-b)

-1.9091639518737793
